<a href="https://colab.research.google.com/github/jsansao/idl/blob/main/Tarefa7_IdentificacaoPoliticos_Dicas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Tarefa 7 - Identificação facial 

Uma agremiação brasileira optou por usar um método de identificação facial para autenticar o cadastro de seus filiados. Infelizmente, a solução foi pouco testada e foi um gargalo no processamento dos cadastros. 

Nessa tarefa, vamos testar os métodos de reconhecimento facial com políticos historicamente vinculados à referida agremiação (alguns não estão mais filiados à mesma). 

Para completar a tarefa, vamos também testar a rede treinada com fotos dos mesmos políticos com máscaras faciais e avaliar acurácia nas predições com as redes originalmente treinadas. 

O dataset foi disponibilizado no github com os conjuntos de treinamento e validação separados. Um segundo conjunto de validação, com máscaras também está disponível no repositório. 

Abaixo, a seqüência para fazer download do dataset e a extração das faces. 

Você deverá: 

1. Escolher um método para implementar a identificação (embedding+SVM ou transfer learning com fine tuning, conforme as lições anteriores) 

2. Treinar a rede, avaliar acurácia de treinamento e validação

3. Avaliar a acurácia para o conjunto das imagens com máscara facial

## Alguns ajustes iniciais

In [ ]:
!pip install mtcnn
!pip install git+https://github.com/jsansao/keras-vggface.git
!pip install keras_applications


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 2.3 MB 6.3 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/jsansao/keras-vggface.git to /tmp/pip-req-build-45rz5t_f
  Running command git clone -q https://github.com/jsansao/keras-vggface.git /tmp/pip-req-build-45rz5t_f
  Created wheel for keras-vggface: filename=keras_vggface-0.6-py3-none-any.whl size=8315 sha256=26d036fb9075f8063053e4e29c565b8d1523ffcb553e706a84781b5474128f19
  Stored in directory: /tmp/pip-ephem-wheel-cache-s6iqu3kk/wheels/96/e5/44/e821709a866ce00a2f4309c59d8387e6756b3ded415b4b0cc6
Successfully built keras-vggface
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 50 kB 5.8 MB/s 


In [ ]:
!wget https://github.com/jsansao/TucanoDataset/archive/refs/heads/main.zip
!unzip main.zip

--2022-12-06 11:36:18--  https://github.com/jsansao/TucanoDataset/archive/refs/heads/main.zip
Resolving github.com (github.com)... 192.30.255.113
Connecting to github.com (github.com)|192.30.255.113|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://codeload.github.com/jsansao/TucanoDataset/zip/refs/heads/main [following]
--2022-12-06 11:36:18--  https://codeload.github.com/jsansao/TucanoDataset/zip/refs/heads/main
Resolving codeload.github.com (codeload.github.com)... 192.30.255.121
Connecting to codeload.github.com (codeload.github.com)|192.30.255.121|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/zip]
Saving to: ‘main.zip’

main.zip                [              <=>   ]  74.78M  17.1MB/s    in 4.4s    

2022-12-06 11:36:22 (17.1 MB/s) - ‘main.zip’ saved [78409321]

Archive:  main.zip
e10dc72508ba64b2c5341f1248e1c94089057ba9
   creating: TucanoDataset-main/
   creating: TucanoDataset-main/masc_val

## Funções para extração das faces e geração do dataset com faces

In [ ]:
# face detection 
from os import listdir
from os.path import isdir
from PIL import Image
from matplotlib import pyplot
from numpy import savez_compressed
from numpy import asarray
import mtcnn

# extract a single face from a given photograph
def extract_face(filename, required_size=(224, 224)):
	# load image from file
	image = Image.open(filename)
	# convert to RGB, if needed
	image = image.convert('RGB')
	# convert to array
	pixels = asarray(image)
	# create the detector, using default weights
	detector = mtcnn.MTCNN()
	# detect faces in the image
	results = detector.detect_faces(pixels)
	# extract the bounding box from the first face
	x1, y1, width, height = results[0]['box']
	# bug fix
	x1, y1 = abs(x1), abs(y1)
	x2, y2 = x1 + width, y1 + height
	# extract the face
	face = pixels[y1:y2, x1:x2]
	# resize pixels to the model size
	image = Image.fromarray(face)
	image = image.resize(required_size)
	face_array = asarray(image)
	return face_array

# load images and extract faces for all images in a directory
def load_faces(directory):
	faces = list()
	# enumerate files
	for filename in listdir(directory):
		# path
		path = directory + filename
		# get face
		face = extract_face(path, (224,224))
		# store
		faces.append(face)
	return faces

# load a dataset that contains one subdir for each class that in turn contains images
def load_dataset(directory):
	X, y = list(), list()
	# enumerate folders, on per class
	for subdir in listdir(directory):
		# path
		path = directory + subdir + '/'
		# skip any files that might be in the dir
		if not isdir(path):
			continue
		# load all faces in the subdirectory
		faces = load_faces(path)
		# create labels
		labels = [subdir for _ in range(len(faces))]
		# summarize progress
		print('>loaded %d examples for class: %s' % (len(faces), subdir))
		# store
		X.extend(faces)
		y.extend(labels)
	return asarray(X), asarray(y)



## Geração dos arquivos com as faces e rótulos

In [ ]:
# load train dataset
trainX, trainy = load_dataset('/content/TucanoDataset-main/train/')
print(trainX.shape, trainy.shape)
# load test dataset
testX, testy = load_dataset('/content/TucanoDataset-main/val/')
# load mask dataset
valX, valy = load_dataset('/content/TucanoDataset-main/masc_val/')

# save arrays to one file in compressed format
savez_compressed('tucano-faces-dataset.npz', trainX, trainy, testX, testy, valX, valy)

1/1 [==============================] - 0s 194ms/step


1/1 [==============================] - 0s 154ms/step


1/1 [==============================] - 0s 152ms/step
>loaded 15 examples for class: FHC
1/1 [==============================] - 0s 145ms/step
>loaded 15 examples for class: JoseAnibal
1/1 [==============================] - 0s 118ms/step
>loaded 15 examples for class: JoaoDoria
1/1 [==============================] - 0s 159ms/step
>loaded 15 examples for class: EduardoLeite
1/1 [==============================] - 0s 117ms/step
>loaded 15 examples for class: JoseSerra
1/1 [==============================] - 0s 119ms/step
>loaded 15 examples for class: ArthurVirgilio
1/1 [==============================] - 0s 110ms/step
>loaded 15 examples for class: GeraldoAlckmin
1/1 [==============================] - 0s 111ms/step
>loaded 15 examples for class: AecioNeves
(120, 224, 224, 3) (120,)
1/1 [==============================] - 0s 116ms/step
>loaded 4 examples for class: FHC
1/1 [==============================] - 0s 295ms/step
>loaded 9 examples for class: JoseAnibal
1/1 [===========================

## Inicie seu código aqui

In [ ]:
# código 

In [ ]:


from numpy import expand_dims
from matplotlib import pyplot
from PIL import Image
from numpy import asarray

import mtcnn
from keras.utils.layer_utils import get_source_inputs

from keras_vggface.vggface import VGGFace
from keras_vggface.utils import preprocess_input
from keras_vggface.utils import decode_predictions


In [ ]:
model = VGGFace(model='resnet50', include_top=False, input_shape=(224, 224, 3), pooling='avg')

94694792/94694792 [==============================] - 4s 0us/step


In [ ]:
def get_embedding(model, face_pixels):
	# scale pixel values
  samples = face_pixels.astype('float32')
  samples = expand_dims(samples, axis=0)
  samples = preprocess_input(samples, version=2)
  # transform face into one sample
  # make prediction to get embedding
  yhat = model.predict(samples)
  return yhat[0]
     

In [ ]:
# convert each face in the train set to an embedding
newTrainX = list()
for face_pixels in trainX:
	embedding = get_embedding(model, face_pixels)
	newTrainX.append(embedding)
newTrainX = asarray(newTrainX)
print(newTrainX.shape)

# convert each face in the test set to an embedding
newTestX = list()
for face_pixels in testX:
	embedding = get_embedding(model, face_pixels)
	newTestX.append(embedding)
newTestX = asarray(newTestX)
print(newTestX.shape)

# convert each face in the val set to an embedding
newValX = list()
for face_pixels in valX:
	embedding = get_embedding(model, face_pixels)
	newValX.append(embedding)
newValX = asarray(newValX)
print(newValX.shape)

1/1 [==============================] - 0s 19ms/step
(120, 2048)
1/1 [==============================] - 0s 24ms/step
(78, 2048)
1/1 [==============================] - 0s 21ms/step
(42, 2048)


In [ ]:
savez_compressed('tucano-faces-embeddings.npz', newTrainX, trainy, newTestX, testy, newValX, valy)

In [ ]:

# develop a classifier for the tucano
from numpy import load
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import Normalizer
from sklearn.svm import SVC
# load dataset
data = load('tucano-faces-embeddings.npz')
trainX, trainy, testX, testy, valX, valy = data['arr_0'], data['arr_1'], data['arr_2'], data['arr_3'], data['arr_4'], data['arr_5']
print('Dataset: train=%d, test=%d, val=%d' % (trainX.shape[0], testX.shape[0], valX.shape[0]))
# normalize input vectors
in_encoder = Normalizer(norm='l2')
trainX = in_encoder.transform(trainX)
testX = in_encoder.transform(testX)
valX = in_encoder.transform(valX)
# label encode targets
out_encoder = LabelEncoder()
out_encoder.fit(trainy)
trainy = out_encoder.transform(trainy)
testy = out_encoder.transform(testy)
valy = out_encoder.transform(valy)
# fit model
model = SVC(kernel='linear', probability=True)
model.fit(trainX, trainy)
# predict
yhat_train = model.predict(trainX)
yhat_test = model.predict(testX)
yhat_val = model.predict(valX)
# score
score_train = accuracy_score(trainy, yhat_train)
score_test = accuracy_score(testy, yhat_test)
score_val = accuracy_score(valy, yhat_val)
# summarize
print('Accuracy: train=%.3f, test=%.3f, val=%.3f' % (score_train*100, score_test*100, score_val*100))
     

Dataset: train=120, test=78, val=42
Accuracy: train=100.000, test=98.718, val=100.000
